<div dir="rtl" style="text-align: right;">
<h2>بررسی راه‌حل</h2>

<p>برای حل این مشکل، ما باید ترکیبی از متدهای NumPy را به کار ببریم تا مشخص کنیم که آیا یک آرایه به عنوان نمای آرایه دیگری است و اگر بله، این نما چگونه تعریف می‌شود.</p>

<p>با استفاده از این روش، ما می‌توانیم با دقت تعیین کنیم که Z2 واقعاً نمایی از Z1 است و پارامترهای مربوط به این نما را تعیین کنیم.</p>
</div>


In [1]:

import numpy as np

# فرض کنید که ما دو آرایه Z1 و Z2 داریم
Z1 = np.arange(0, 100, 2)  # آرایه از 0 تا 100 با گام 2
Z2 = Z1[5:20]  # برشی از Z1

# محاسبه شاخص شروع، پایان و گام
step = Z2.strides[0] // Z1.strides[0]
start_index = (Z2.__array_interface__['data'][0] - Z1.__array_interface__['data'][0]) // Z1.itemsize
end_index = start_index + len(Z2) * step

print("Start Index:", start_index)
print("End Index:", end_index)
print("Step:", step)

# استفاده از np.allclose برای اطمینان از صحت نما
assert np.allclose(Z1[start_index:end_index:step], Z2)


Start Index: 5
End Index: 20
Step: 1


In [2]:
import numpy as np



def find_index(base, view):
    """
    با توجه به یک آرایه که نمایی از یک پایه است، شاخصی پیدا کنید به طوری که پایه[اندیس] نما است
    """

    if not isinstance(view, np.ndarray):
        return "..."

    itemsize = view.itemsize
    
    # با استفاده از روش byte_bound، اشاره‌گر شروع و پایان آرایه‌ها را پیدا کنید
    offset_start = (np.byte_bounds(view)[0] - np.byte_bounds(base)[0])//itemsize
    offset_stop = (np.byte_bounds(view)[-1] - np.byte_bounds(base)[-1]-1)//itemsize
    
    # شاخص‌های شروع و توقف را از فاصله‌ها محاسبه کنید  
    index_start = np.unravel_index(offset_start, base.shape)
    index_stop = np.unravel_index(base.size+offset_stop, base.shape)
    
    # از خاصیت strides برای پیدا کردن تعداد بایت‌هایی که برای رفتن از یک عنصر به عنصر دیگر نیاز است استفاده کنید 
    index_step  = np.array(view.strides)//np.array(base.strides)

    index = ""
    for i in range(len(index_step)):
        start = index_start[i]
        stop = index_stop[i]
        step = index_step[i]

        if stop == start:
            stop, step = None, None
        else:
            if stop == base.shape[i] - 1:
                stop = None
            else:
                stop = stop
            if start == 0:
                start = None
        if step is not None and stop is not None:
            if step < 0:
                start, stop = stop, start - 1
            else:
                start, stop = start, stop + 1
            
        if start is not None:
            index += str(start)
        if stop is not None:
            index += ":" + str(stop)
        elif step is not None:
            index += ":"
        if step is not None:
            index += ":" + str(step)
        index += ','
    index = index[:-1]

    return index



if __name__ == '__main__':

    base = np.arange(8*8).reshape(8,8)

    # زیر-آرایه
    Z = base[1:-1,1:-1]
    index = find_index(base,Z)
    print(np.allclose(Z, eval("base[%s]" % index)))
    
    # دو آیتم در میان
    Z = base[::2,::2]
    index = find_index(base,Z)
    print(np.allclose(Z, eval("base[%s]" % index)))
    
    # ستون اول
    Z = base[:,0]
    index = find_index(base,Z)
    print(np.allclose(Z, eval("base[%s]" % index)))
    
    # ردیف اول
    Z = base[0,:]
    index = find_index(base,Z)
    print(np.allclose(Z, eval("base[%s]" % index)))
    
    # معکوس جزئی
    Z = base[4:1:-1,6:2:-1]
    index = find_index(base,Z)
    print(np.allclose(Z, eval("base[%s]" % index)))
    
    # # معکوس کامل
    Z = base[::-1,::-1]
    index = find_index(base,Z)
    print(np.allclose(Z, eval("base[%s]" % index)))

    # تصادفی
    Z = base[1:5:3,3:1:-1]
    index = find_index(base,Z)
    print(np.allclose(Z, eval("base[%s]" % index)))

True
True
True
True
True
True
True
